![](./figures/Logo.PNG)

Please click the <span>&#x23E9;</span> button to run all cells before you start working on the notebook ...

## In this part of the tutorial, you will
* use regional sensitivity analysis (RSA)

---

# 4 a - Regional sensitivity analysis (RSA)

---

## 1 Introduction

Regional sensitivity analysis evaluates how variations in input parameters impact the outcomes of a model. 

Today, we use statistical metrics to distinguish between parameter sets that produce behavioral and non-behavorial simulation output.
We apply the same metrics to implement and assess Generalized Likelihood Uncertainty Estimation (GLUE).

## 2 Loading Catchment Dat

In [1]:
import sys
sys.path.append('src/')
import HBV
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from numbers import Number
import RSA_thres as RSA           # module to perform RSA with threshold
import plot_functions as SA_plot  # module to visualize the results
from sampling import AAT_sampling # module to perform the input sampling
from ipywidgets import interact, interactive_output, Dropdown, Checkbox, VBox, Layout, fixed, DatePicker

In [2]:
def rmse(obs, sim, spinup=365):
    obs = obs[spinup:]
    sim = sim[spinup:]
    return np.sqrt(np.mean(np.square(np.subtract(obs, sim))))

def abs_bias(obs, sim, spinup=365):
    obs = obs[spinup:]
    sim = sim[spinup:]
    return np.mean(np.abs(sim - obs))

#def nse(obs, sim, spinup=365):
#    obs = obs[spinup:]
#    sim = sim[spinup:]
#    r_nse     = np.corrcoef(obs, sim)[0][1] 
#    alpha_nse = np.divide(np.std(sim), np.std(obs))
#    beta_nse  = np.divide(np.subtract(np.mean(sim), np.mean(obs)), np.std(obs))
#    return 2 * alpha_nse * r_nse - np.square(alpha_nse) - np.square(beta_nse)

def hbv(par, precip, temp, evap):
    # Run HBV snow routine
    p_s, _, _ = HBV.snow_routine(par[:4], temp, precip)
    # Run HBV runoff simulation
    Case = 1 # for now we assume that the preferred path in the upper zone is runoff (Case = 1), it can be set to percolation (Case = 2)
    ini = np.array([0,0,0]) # initial state
    runoff_sim, _, _ = HBV.hbv_sim(par[4:], p_s, evap, Case, ini)
    return runoff_sim

In [3]:
# DO NOT ALTER! code to select the catchment

catchment_names = ["Medina River, TX, USA", "Siletz River, OR, USA", "Trout River, BC, Canada"]
dropdown = Dropdown(
    options=catchment_names,
    value=catchment_names[0],
    description='Catchment:',
    disabled=False
)

display(dropdown)

Dropdown(description='Catchment:', options=('Medina River, TX, USA', 'Siletz River, OR, USA', 'Trout River, BC…

In [4]:
# read catchment data
catchment_name = dropdown.value
file_dic = {catchment_names[0]: "camels_08178880", catchment_names[1]: "camels_14305500", catchment_names[2]: "hysets_10BE007"}
df_obs = pd.read_csv(f"data/{file_dic[catchment_name]}.csv")

# correctly load the date and restrict analysis to one year
df_obs.date = pd.to_datetime(df_obs['date'], format='%Y-%m-%d')
start_date  = '2003-01-01' # the first year is used as spinup
end_date    = '2008-12-30'

# Index frame by date
df_obs.set_index('date', inplace=True)
# Select time frame
df_obs = df_obs[start_date:end_date]
# Reformat the date for plotting
df_obs["date"] = df_obs.index.map(lambda s: s.strftime('%b-%d-%y'))
# Reindex
df_obs = df_obs.reset_index(drop=True)
# Select snow, precip, PET, streamflow and T
df_obs = df_obs[["snow_depth_water_equivalent_mean", "total_precipitation_sum","potential_evaporation_sum","streamflow", "temperature_2m_mean", "date"]]
# Rename variables
df_obs.columns = ["Snow [mm/day]", "P [mm/day]", "PET [mm/day]", "Q [mm/day]", "T [C]", "Date"]

# load calibrated parameters
params_calibrated = pd.read_csv("./data/calibrated_parameters - HBV.csv")
params_calibrated = params_calibrated[(params_calibrated.catchment_name == catchment_name) & (params_calibrated.objective_function == "nse")] # use only this catchment and the rmse parameters
params_calibrated = params_calibrated.iloc[0,3:].values

## 3 Behavioral and Non-Behavioral Parameters

In [5]:
param_names = ["Ts", "CFMAX", "CFR", "CWH", "BETA", "LP", "FC", "PERC", "K0", "K1", "K2", "UZL", "MAXBAS"]
lower       = np.array([-3, 0, 0, 0, 0, 0.3, 1, 0, 0.05, 0.01, 0.005, 0, 1])
upper       = np.array([3, 20, 1, 0.8, 7, 1, 2000, 100, 2, 1, 0.1, 100, 6])

#ranges = list(zip(params_calibrated, lower, upper))
#lower  = np.array([max(low, value - 0.1*(high - low)) for value, low, high in ranges])
#upper  = np.array([min(value + 0.1*(high - low), high) for value, low, high in ranges])

In [6]:
@interact(N=(50, 500, 10))
def sample_and_run(N=300):

    # SAMPLE PARAMETER SPACE AND RUN HBV
    np.random.seed(1234)

    global samples, errors
    
    # take N samples using a latin-hypercube strategy for the value ranges with uniform marginal distributions
    samples = AAT_sampling("lhs", len(param_names), sp.stats.uniform, np.array([lower, upper - lower]).T.tolist(), N=N)
    
    # evaluate HBV for these parameters
    errors = np.zeros((N, 2))
    for i in range(N):
        Q_sim = hbv(samples[i], df_obs["P [mm/day]"], df_obs["T [C]"], df_obs["PET [mm/day]"])
        errors[i, 0] = abs_bias(df_obs["Q [mm/day]"], Q_sim) # calculate  nse for this paramter set
        errors[i, 1] =     rmse(df_obs["Q [mm/day]"], Q_sim) # calculate rmse for this paramter set 

    @interact(bias_threshold=(errors[:,0].min(), errors[:,0].max(), 0.005), rmse_threshold=(errors[:,1].min(), errors[:,1].max(), 0.005), zoom=(0, 0.5, 0.05), solution=False)
    def plot_behavioral(bias_threshold=np.quantile(errors[:,0], 0.2), rmse_threshold=np.quantile(errors[:,1], 0.2), zoom=0, solution=False):
        
        # which paramters are behavioral
        behavioral = (errors[:,0] <= bias_threshold) & (errors[:,1] <= rmse_threshold)
        
        # PLOT
        fig, axs = plt.subplots(2, 5, figsize=(4*5, 4*2))

        for i, metric in enumerate(["|Bias|", "RMSE"]):
            
            for j, param in enumerate(["CWH", "BETA", "PERC", "UZL"]):

                # index of the paramter value
                k = param_names.index(param)
                
                ax = axs[i, j]
                if i == 0: ax.set_title(param)
                ax.set_xlabel(param)
                ax.set_ylabel(metric)
                
                # scatterplot for the metric values
                ax.scatter(samples[:,k], errors[:,i], c=np.where(behavioral, "blue", "red"), alpha=0.75)

                # rugplot of the behavioral paramters
                ax.plot(samples[behavioral,k], errors[:,i].min() - 0.01 + np.zeros_like(samples[behavioral,k]), '|', color='blue', alpha=0.5) 

                # zoom into plot if set
                ax.set_ylim([errors[:,i].min() - 0.02, ax.get_ylim()[1]*(1 - zoom)])

                # horizontal line for the threshold
                threshold = [bias_threshold, rmse_threshold][i]
                if threshold < ax.get_ylim()[1]:
                    ax.axhline(threshold, color="black", ls="--")
                    label = ax.text(samples[:,k].min(), threshold, f"{metric} = {threshold:.2f}", ha="left", va="center", fontsize=10)
                    label.set_bbox(dict(facecolor="white"))
            
        ax = axs[0, -1]
        cnts = np.unique(np.where(behavioral, "Behavioral", "Non-Behavioral"), return_counts=True)
        bars = ax.bar(*cnts)
        ax.set_ylim([0, ax.get_ylim()[1] + 10])
        ax.set_title("Fraction of Behavioral Parameters")
        for i, (key, bar) in enumerate(zip(cnts[0], bars)):
            bar.set_color("blue" if key == "Behavioral" else "red")
            ax.text(bar.xy[0] + bar.get_width()/2, bar.get_height(), f"{bar.get_height()/N:.0%}", ha="center", va="bottom")

        ax = axs[1, -1]
        if solution:
            ax.scatter(errors[:,0], errors[:,1], color=np.where(behavioral, "blue", "red"))
            ax.set_xlabel("|Bias|")
            ax.set_ylabel("RMSE")
            ax.set_title("Bias - RMSE Scatterplot")
            ax.set_xlim([errors[:,0].min() - 0.02, ax.get_xlim()[1]*(1 - zoom)])
            ax.set_ylim([errors[:,1].min() - 0.02, ax.get_ylim()[1]*(1 - zoom)])

            if bias_threshold < ax.get_xlim()[1]:
                ax.axvline(bias_threshold, color="black", ls="--")
                label = ax.text(bias_threshold, np.mean(ax.get_ylim()), f"|Bias| = {bias_threshold:.2f}", ha="center", va="center", fontsize=10)
                label.set_bbox(dict(facecolor="white"))
            if rmse_threshold < ax.get_ylim()[1]:
                ax.axhline(rmse_threshold, color="black", ls="--")
                label = ax.text(np.mean(ax.get_xlim()), rmse_threshold, f"RMSE = {rmse_threshold:.2f}", ha="center", va="center", fontsize=10)
                label.set_bbox(dict(facecolor="white"))
        else:
            ax.axis("off")

        plt.tight_layout()
        plt.show()

interactive(children=(IntSlider(value=300, description='N', max=500, min=50, step=10), Output()), _dom_classes…

<div style="background:#e0f2fe;border:1mm solid SkyBlue; padding:1%">
    <h4><span>&#129300 </span>Task I: Thresholds</h4>
    We have again run HBV for N different paramter combinations that come from a latin-hypercube strategy. In the scatterplots above you can see the absolute Bias and RMSE values for these runs. You can now select thresholds for the metrics. Once both metrics are below their threshold, a parameter set is marked as behavioral. On the bottom of the plot you see what is called a rugplot. Each tick indicates one behavioral parameter set. As you can see, we have omitted some parameters so that the plot is not too large.
    <ol>
        <li>How does changing the thresholds affect the distribution of behavioral parameters?</li>
        <li>Why do you see some non-behavioral points that lie above the horizontal line that indices the threshold for a metric?</li>
        <li>How do the thresholds look in a Bias-RMSE scatterplot? Where would you find the bahavioral parameters? Is there a better way to define a threshold?</li>
    </ol>
    You can show the solution to the third question by activating the solution checkbox.
</div>

_PUT YOUR ANSWER HERE_

<div style="background:#e0f2fe;border:1mm solid SkyBlue; padding:1%">
</div>

In [7]:
@interact(bias_threshold=(errors[:,0].min(), errors[:,0].max(), 0.005), rmse_threshold=(errors[:,1].min(), errors[:,1].max(), 0.005))
def plot_behavioral(bias_threshold=np.quantile(errors[:,0], 0.2), rmse_threshold=np.quantile(errors[:,1], 0.2)):
    
    # which paramters are behavioral
    behavioral = (errors[:,0] <= bias_threshold) & (errors[:,1] <= rmse_threshold)
    
    # PLOT
    fig, axs = plt.subplots(1, 4, figsize=(4*4, 4*1), sharex="col", sharey=True)

    for j, param in enumerate(["CWH", "BETA", "PERC", "UZL"]):

        # index of the paramter value
        k = param_names.index(param)
        
        ax = axs[j]
        ax.set_xlabel(param)
        ax.set_ylabel("Probability")
        ax.set_xlim(samples[:,k].min(), samples[:,k].max())
        
        # CDF plots
        if behavioral.sum() > 1:
            ax.ecdf(samples[ behavioral,k], color="blue", label="Behavioral")
        if behavioral.sum() < len(behavioral):
            ax.ecdf(samples[~behavioral,k], color="red", label="Non-Behavioral")

        # uniform distribution
        ax.plot(ax.get_xlim(), [0, 1], color="gray", zorder=0, label="Uniform")

        # rugplot of the behavioral paramters
        ax.plot(samples[ behavioral,k], 0.01 + np.zeros_like(samples[ behavioral,k]), '|', color='blue', alpha=0.5) 
        ax.plot(samples[~behavioral,k], 0.99 + np.zeros_like(samples[~behavioral,k]), '|', color='red',  alpha=0.5) 

        
    plt.tight_layout()
    plt.show()

interactive(children=(FloatSlider(value=0.36870101321509546, description='bias_threshold', max=1.4772442719929…

<div style="background:#e0f2fe;border:1mm solid SkyBlue; padding:1%">
    <h4><span>&#129300 </span>Task II: Cumulative Distribution Functions</h4>
    Above you see the CDFs for the behavioral and non-behavioral parameter sets.
    <ol>
        <li>How does changing the thresholds affect the CDF of behavioral and non-behavioral parameters?</li>
        <li>How can you read the values of well-performing parameter sets from the plot?</li>
    </ol>
</div>

_PUT YOUR ANSWER HERE_

<div style="background:#e0f2fe;border:1mm solid SkyBlue; padding:1%">
</div>

## 4 GLUE Algorithm for Significance Estimates

In [8]:
def weighted_percentile(data, weights, perc):
    ix = np.argsort(data)
    data = data[ix] # sort data
    weights = weights[ix] # sort weights
    cdf = (np.cumsum(weights) - 0.5 * weights) / np.sum(weights) # 'like' a CDF function
    return np.interp(perc, cdf, data)

In [9]:
Q_sim_all = np.stack([hbv(sample, df_obs["P [mm/day]"], df_obs["T [C]"], df_obs["PET [mm/day]"]) for sample in samples])
# rerun this cell when you changed the number of paramter sets for the first task

In [10]:
@interact(bias_threshold=(errors[:,0].min(), errors[:,0].max(), 0.005), rmse_threshold=(errors[:,1].min(), errors[:,1].max(), 0.005))
def plot_behavioral(bias_threshold=np.quantile(errors[:,0], 0.2), rmse_threshold=np.quantile(errors[:,1], 0.2)):

    # only use behavioral samples
    behavioral = (errors[:,0] <= bias_threshold) & (errors[:,1] <= rmse_threshold)
    Q_sim = Q_sim_all[behavioral]
    Q_obs = np.tile(df_obs["Q [mm/day]"], (Q_sim.shape[0], 1))

    # calculate likelihood of model run
    likelihood = 1/np.var(Q_sim - Q_obs, axis=1)
    
    Q_sim_lower = np.apply_along_axis(lambda x: weighted_percentile(x, likelihood, 0.05), axis=0, arr=Q_sim)
    Q_sim_upper = np.apply_along_axis(lambda x: weighted_percentile(x, likelihood, 0.95), axis=0, arr=Q_sim)
    Q_sim_mean  = Q_sim.mean(axis=0)

    # plot the GLUE approximation
    plt.figure(figsize=(20, 4))
    plt.fill_between(df_obs.Date, Q_sim_lower, Q_sim_upper, color="black", alpha=0.2, label="90% CI")
    plt.plot(df_obs.Date, Q_sim_mean, color="black", label="Ensemble Mean")
    plt.plot(df_obs.Date, Q_obs[0], color="red", label="Observed")
    plt.xlim(df_obs.Date.iloc[4*365], df_obs.Date.iloc[5*365])
    plt.gca().xaxis.set_major_locator(plt.matplotlib.dates.MonthLocator())
    plt.title(catchment_name)
    plt.xlabel("Date")
    plt.ylabel("Runoff [mm/day]")
    plt.legend()
    
    ax = plt.twinx()
    ax.invert_yaxis()
    ax.set_ylabel("Precipitation [mm/day]")
    ax.bar(df_obs.Date, df_obs["P [mm/day]"], color="C0", label="Precipitation", zorder=0)
    ax.set_ylim([df_obs["P [mm/day]"].max()*1.5, 0])
    plt.legend()
    
    plt.show()

interactive(children=(FloatSlider(value=0.36870101321509546, description='bias_threshold', max=1.4772442719929…

<div style="background:#e0f2fe;border:1mm solid SkyBlue; padding:1%">
    <h4><span>&#129300 </span>Task III: GLUE</h4>
    In the above plot, you the a confidence interval estimate for the model runs using the GLUE method. It works by taking an average over the simualte runoff only for behavioral parameter sets. The likelihood for each parameter set is determined by how well the fit is.
    <ol>
        <li>How does the confidence interval change, when you alter the thresholds for absolute bias and RMSE?</li>
        <li>Can you explain in your own words how the GLUE algorithm works?</li>
    </ol>
    As you can see, the fit between the GLUE ensemble and the observed runoff is quite poor. We suspect that this is due to problems in the HBV model.
    <ol start="3">
        <li>Why could that be the case? Which misrepresentations of physical processes could lead to this mismatch between model results and observed runoff?</li>
    </ol>
</div>

_PUT YOUR ANSWER HERE_

<div style="background:#e0f2fe;border:1mm solid SkyBlue; padding:1%">
</div>